In [1]:
import numpy as np

height = 270
width = 480
fps = 30

# read data as unsigned bytes
raw_bytes = np.fromfile("./dataset/Videos/data_test1.rgb", np.dtype('B'))

n_frames = len(raw_bytes)//(height*width*3)
frames = raw_bytes.reshape((n_frames, 3, height, width))
frames = np.moveaxis(frames, 1, -1)  # pack rgb values per pixel

In [2]:
frames[0][0]  # first row of the first frame

array([[248, 253, 255],
       [249, 254, 255],
       [249, 254, 255],
       ...,
       [  0,   0,   0],
       [  0,   0,   0],
       [  0,   0,   0]], dtype=uint8)

In [3]:
import ffmpeg

dest = "./dataset/Videos/data_test1.avi"

# convert rgb frames to standard video format
process = (
    ffmpeg
    .input('pipe:', format='rawvideo', pix_fmt='rgb24', s='{}x{}'.format(width, height), framerate=fps)
    .output(dest, pix_fmt='yuv420p', vcodec="libx264")
    .overwrite_output()
    .run_async(pipe_stdin=True, overwrite_output=True, pipe_stderr=True)
)

for frame in frames:
    process.stdin.write(frame.astype(np.uint8).tobytes())
    
process.stdin.close()
process.wait()

0

In [4]:
# join video and audio
input_video = ffmpeg.input("./dataset/Videos/data_test1.avi")
input_audio = ffmpeg.input("./dataset/Videos/data_test1.wav")

dest = "./dataset/Videos/data_test1_cmb.avi"
ffmpeg.output(input_video, input_audio, dest, pix_fmt='yuv420p', vcodec="libx264").overwrite_output().run()

ffmpeg version 5.0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with Apple clang version 13.1.6 (clang-1316.0.21.2)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/5.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox

(None, None)